In [7]:
# ============ 完整的单个单元格版本 ============
# 在 Jupyter 中直接运行这个单元格
import random
import asyncio
import json
import os
import re
import time
from pathlib import Path
from datetime import datetime
import pandas as pd
from playwright.async_api import async_playwright

APPLE_HOME = "https://www.apple.com/jp/"
CSV_PATH = "../some-data/Book10.csv"  # 修改为你的实际路径
AUTH_CACHE_FILE = ".auth_cache.json"
DEBUG_DIR = "pw_debug"
BTN_UPDATE_PICKUP = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updatePickupContact"
INP_LASTNAME = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.lastName"
INP_FIRSTNAME = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.firstName"
BTN_SUBMIT = "orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.pickup-contact-submit"
print("✅ 常量定义完成")
print(f"  编辑按钮 ID: {BTN_UPDATE_PICKUP}")
print(f"  姓输入框 ID: {INP_LASTNAME}")
print(f"  名输入框 ID: {INP_FIRSTNAME}")
print(f"  提交按钮 ID: {BTN_SUBMIT}")

import re

def split_name_jp(fullname: str):
    """将 '神原　龍一' / '神原 龍一' 拆成 (last, first)"""
    if not isinstance(fullname, str) or not fullname.strip():
        return "", ""

    # 把常见“空白类字符”统一成普通空格
    s = fullname.replace("\u3000", " ")  # 全角空格
    s = re.sub(r"\s+", " ", s).strip()   # 所有空白压成一个空格

    parts = s.split(" ")
    print("split_name_jp :", parts)

    if len(parts) >= 2:
        return parts[0], parts[1]
    return parts[0], ""



async def load_auth_cache(apple_id: str) -> dict | None:
    """加载认证缓存"""
    if not Path(AUTH_CACHE_FILE).exists():
        return None

    try:
        with open(AUTH_CACHE_FILE, "r") as f:
            cache = json.load(f)

        if cache.get("apple_id") == apple_id:
            cache_time = cache.get("timestamp", 0)
            if datetime.now().timestamp() - cache_time < 12 * 3600:
                print(f"✅ 使用缓存的认证信息")
                return cache
    except:
        pass

    return None


async def save_auth_cache(apple_id: str, cookies: list):
    """保存认证缓存"""
    try:
        cache_data = {
            "apple_id": apple_id,
            "cookies": [
                {
                    "name": c["name"],
                    "value": c["value"],
                    "domain": c.get("domain"),
                    "path": c.get("path"),
                }
                for c in cookies if c.get("name")
            ],
            "timestamp": datetime.now().timestamp(),
        }

        with open(AUTH_CACHE_FILE, "w") as f:
            json.dump(cache_data, f, indent=2)

        print(f"💾 认证信息已保存")
    except Exception as e:
        print(f"⚠️  保存缓存失败: {e}")


async def maybe_switch_to_popup_or_same_page(context, current_page, click_coro, wait_ms=1200):
    """检查是否打开新页面"""
    before = len(context.pages)
    await click_coro
    await current_page.wait_for_timeout(wait_ms)
    after = len(context.pages)

    return context.pages[-1] if after > before else current_page


async def open_orders_page(page, context):
    """打开订单页面"""
    print("🔄 打开订单页面...")

    await page.goto(APPLE_HOME, wait_until="domcontentloaded", timeout=30000)
    await page.locator("#globalnav-menubutton-link-bag").wait_for(timeout=10000)
    await page.locator("#globalnav-menubutton-link-bag").click()

    bag_menu = page.locator("#globalnav-submenu-bag")
    orders_link = bag_menu.get_by_role(
        "link",
        name=re.compile(r"(ご注文|注文|Orders|注文状況|ご注文状況)")
    ).first

    await orders_link.wait_for(timeout=15000)

    orders_page = await maybe_switch_to_popup_or_same_page(
        context, page, orders_link.click(), wait_ms=1500
    )

    await orders_page.wait_for_load_state("domcontentloaded", timeout=30000)
    print(f"✅ 订单页已打开")

    return orders_page


async def sign_in_if_needed(page, apple_id: str, password: str):
    """登录流程"""
    print(f"🔐 检查登录状态...")

    await page.wait_for_load_state("domcontentloaded", timeout=30000)

    # 尝试使用缓存
    # auth_cache = await load_auth_cache(apple_id)
    # if auth_cache and auth_cache.get("cookies"):
    #     try:
    #         await page.context.add_cookies(auth_cache["cookies"])
    #         await page.reload(wait_until="domcontentloaded", timeout=30000)
    #
    #         frame_loc = page.frame_locator("iframe").first
    #         account_field = frame_loc.locator("#account_name_text_field")
    #
    #         try:
    #             await account_field.wait_for(timeout=3000)
    #             print(f"⚠️  缓存认证已失效")
    #         except:
    #             print(f"✅ 缓存认证有效，已登录")
    #             return
    #     except:
    #         pass

    # 检查是否需要登录
    frame_loc = page.frame_locator("iframe").first
    account_field = frame_loc.locator("#account_name_text_field")

    try:
        await account_field.wait_for(timeout=6000)
        print(f"📝 需要登录")
    except:
        print(f"✅ 已登录（无登录框）")
        return

    # 执行登录
    try:
        print(f"  输入 Apple ID...")
        await account_field.fill(apple_id)
        await frame_loc.locator("#sign-in").click()
        time.sleep(random.randint(0,20))
        print(f"  输入密码...")
        password_field = frame_loc.locator("#password_text_field")
        await password_field.wait_for(timeout=15000)
        await password_field.fill(password)
        time.sleep(random.randint(0,20))

        await frame_loc.locator("#sign-in").click()

        print(f"⏳ 等待登录完成...")

        try:
            await page.wait_for_selector("#rs-container", timeout=180000)
            print(f"✅ 登录成功")
        except:
            print(f"⏳ 可能需要 2FA 验证...")
            try:
                await page.wait_for_selector("#rs-container", timeout=120000)
                print(f"✅ 登录成功（2FA 完成）")
            except:
                input("❓ 请确认已完成登录，按回车继续：")

        cookies = await page.context.cookies()
        await save_auth_cache(apple_id, cookies)

    except Exception as e:
        print(f"❌ 登录失败: {e}")
        raise


async def open_target_order(page, ordernumber: str | None = None, product_fallback: str = "iPhone 17 Pro"):
    """打开订单"""
    print(f"📦 打开订单...")

    await page.wait_for_load_state("domcontentloaded", timeout=30000)

    if ordernumber and isinstance(ordernumber, str) and ordernumber.strip():
        on = ordernumber.strip()
        print(f"  搜索订单号: {on}")

        try:
            link = page.get_by_role("link", name=re.compile(re.escape(on))).first
            await link.wait_for(timeout=10000)
            await link.click()
            time.sleep(random.randint(0,10))
            await page.wait_for_load_state("domcontentloaded", timeout=30000)
            return
        except:
            pass

        try:
            txt = page.get_by_text(on).first
            await txt.wait_for(timeout=10000)
            await txt.click()
            await page.wait_for_load_state("domcontentloaded", timeout=30000)
            return
        except:
            pass

        print(f"⚠️  订单号未找到，使用商品名")

    print(f"  使用商品名: {product_fallback}")
    prod_link = page.get_by_role("link", name=re.compile(re.escape(product_fallback))).first
    await prod_link.wait_for(timeout=15000)
    await prod_link.click()
    time.sleep(random.randint(0,10))
    await page.wait_for_load_state("domcontentloaded", timeout=30000)
    print(f"✅ 订单已打开")


async def update_pickup_contact(page, new_last: str, new_first: str):
    """
    更新取货联系人 - 修复版

    从完整姓名中分离姓和名
    例如: "神原 龍一" -> 姓: "神原", 名: "龍一"
    """

    print(f"   姓: {new_last}, 名: {new_first}")

    try:
        await page.wait_for_load_state("networkidle", timeout=30000)
        await page.wait_for_timeout(2000)

        # ========== 点击编辑按钮 ==========
        print(f"  🔍 找编辑按钮...")
        update_btn = page.locator(f"//*[@id='{BTN_UPDATE_PICKUP}']")
        await update_btn.wait_for(state="visible", timeout=20000)
        print(f"  ✅ 找到编辑按钮")
        await update_btn.click()
        print(f"  ✓ 已点击编辑按钮")
        await page.wait_for_timeout(2000)

        # ========== 填充姓 ==========
        print(f"  📝 填充姓: {new_last}")
        last_name_input = page.locator(f"//*[@id='{INP_LASTNAME}']")
        await last_name_input.wait_for(state="visible", timeout=15000)
        time.sleep(random.randint(0,20))
        print(f"  ✅ 找到姓输入框")
        await last_name_input.clear()
        await page.wait_for_timeout(300)
        await last_name_input.type(new_last, delay=50)
        print(f"  ✓ 已填充姓: {new_last}")
        await page.wait_for_timeout(500)
        time.sleep(random.randint(0,20))

        # ========== 填充名 ==========
        print(f"  📝 填充名: {new_first}")
        first_name_input = page.locator(f"//*[@id='{INP_FIRSTNAME}']")
        await first_name_input.wait_for(state="visible", timeout=15000)
        time.sleep(random.randint(0,20))
        print(f"  ✅ 找到名输入框")
        await first_name_input.clear()
        await page.wait_for_timeout(300)
        await first_name_input.type(new_first, delay=50)
        time.sleep(random.randint(0,20))
        print(f"  ✓ 已填充名: {new_first}")
        await page.wait_for_timeout(500)

        # ========== 提交 ==========
        print(f"  🚀 提交...")
        submit_btn = page.locator(f"//*[@id='{BTN_SUBMIT}']")
        await submit_btn.wait_for(state="visible", timeout=15000)
        time.sleep(random.randint(0,20))
        print(f"  ✅ 找到提交按钮")
        await submit_btn.click()
        print(f"  ✓ 已提交")
        await page.wait_for_timeout(3000)

        print(f"✅ 联系人信息已更新\\n")

    except Exception as e:
        print(f"\\n❌ 更新失败: {e}\\n")
        try:
            await page.screenshot(path="debug_error.png")
            print(f"📸 调试截图: debug_error.png\\n")
        except:
            pass
        raise


async def sign_out(page):
    """登出"""
    print(f"🚪 登出...")

    try:
        await page.locator("#globalnav-menubutton-link-bag").wait_for(timeout=10000)
        time.sleep(random.randint(0,20))
        await page.locator("#globalnav-menubutton-link-bag").click()

        bag_menu = page.locator("#globalnav-submenu-bag")
        signout_link = bag_menu.get_by_role(
            "link",
            name=re.compile(r"(サインアウト|Sign Out|ログアウト)")
        ).first

        try:
            await signout_link.wait_for(timeout=8000)
            await signout_link.click()
            print(f"✅ 已登出\\n")
        except:
            print(f"⚠️  无登出选项（非关键）\\n")

    except Exception as e:
        print(f"⚠️  登出失败（非关键）: {e}\\n")


async def run_one_row(browser, row: dict):
    """处理一行"""

    apple_id = str(row.get("apple-email", "")).strip()
    password = str(row.get("pass", "")).strip()
    ordernumber = str(row.get("ordernumber", "")).strip() if row.get("ordernumber") is not None else ""
    newname = str(row.get("newname", "")).strip()

    new_last, new_first = split_name_jp(newname)

    # 参数验证
    if not apple_id:
        raise ValueError("缺少 apple-email 列")

    if not password:
        from getpass import getpass
        password = getpass(f"请输入 {apple_id} 的密码：")

    if not new_last:
        raise ValueError("缺少 newname 或无法解析")

    context = await browser.new_context(
        locale="ja-JP",
        viewport={"width": 1280, "height": 840},
    )

    page = await context.new_page()
    page.set_default_timeout(30000)

    try:
        print(f"\\n{'='*70}")
        print(f"👤 {apple_id} | 订单: {ordernumber or '(商品名)'} | 名字: {new_last} {new_first}")
        print(f"{'='*70}")

        orders_page = await open_orders_page(page, context)
        await sign_in_if_needed(orders_page, apple_id, password)
        time.sleep(random.randint(0,5))
        await open_target_order(orders_page, ordernumber or None)
        time.sleep(random.randint(0,5))
        await update_pickup_contact(orders_page, new_last, new_first)
        time.sleep(random.randint(0,5))
        await sign_out(orders_page)

        print(f"✅ [OK] {apple_id}\\n")
        return True

    except Exception as e:
        os.makedirs(DEBUG_DIR, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        shot = f"{DEBUG_DIR}/fail_{re.sub(r'[^a-zA-Z0-9]+', '_', apple_id)[:30]}_{timestamp}.png"

        try:
            await page.screenshot(path=shot, full_page=True)
            print(f"📸 {shot}\\n")
        except:
            pass

        print(f"❌ [FAIL] {apple_id}: {str(e)}\\n")
        return False

    finally:
        await context.close()


async def run_batch(csv_path: str, headless: bool = True):
    """批处理"""

    print(f"\\n{'='*70}")
    print(f"📋 CSV: {csv_path}")

    if not Path(csv_path).exists():
        print(f"❌ 文件不存在\\n")
        return

    df = pd.read_csv(csv_path)
    rows = df.to_dict(orient="records")

    print(f"📊 总计: {len(rows)} 行")
    print(f"{'='*70}\\n")

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=headless,
            slow_mo=80,
            args=[
                "--disable-blink-features=AutomationControlled",
                "--disable-dev-shm-usage",
            ]
        )

        success_count = 0
        fail_count = 0

        try:
            for idx, row in enumerate(rows, 1):
                try:
                    print(f"[{idx}/{len(rows)}]")
                    time.sleep(random.randint(0,200))
                    success = await run_one_row(browser, row)

                    if success:
                        success_count += 1
                    else:
                        fail_count += 1

                    if idx < len(rows):
                        await asyncio.sleep(2)

                except Exception as e:
                    fail_count += 1
                    print(f"❌ 错误: {str(e)}\\n")
                    await asyncio.sleep(3)

        finally:
            await browser.close()

            print(f"\\n{'='*70}")
            print(f"✅ 成功: {success_count}/{len(rows)}")
            print(f"❌ 失败: {fail_count}/{len(rows)}")
            print(f"{'='*70}\\n")


# ============ 执行 ============

print("\\n开始运行...\\n")

# 确认 CSV 路径
csv_file = CSV_PATH
print(f"CSV: {csv_file}")
print(f"存在: {Path(csv_file).exists()}\\n")

if Path(csv_file).exists():
    # 显示 CSV 前几行
    df_preview = pd.read_csv(csv_file)
    print("📋 CSV 预览:")
    print(df_preview.head())
    print()

    # 执行
    await run_batch(csv_file, headless=True)
else:
    print(f"❌ 请修改 CSV_PATH")

✅ 常量定义完成
  编辑按钮 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updatePickupContact
  姓输入框 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.lastName
  名输入框 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.update-pickup-contact.firstName
  提交按钮 ID: orderDetail.orderItems.orderItem-0000101.shippingInfo.updateThirdPartyPickupInfo.pickup-contact-submit
\n开始运行...\n
CSV: ../some-data/Book10.csv
存在: True\n
📋 CSV 预览:
         apple-email      pass  ordernumber oldname newname
0  j5x648@icloud.com  Qp272727  W1610977865   高橋 光晴   神原　龍一

\n======================================================================
📋 CSV: ../some-data/Book10.csv
📊 总计: 1 行
======================================================================\n
[1/1]
split_name_jp : ['神原', '龍一']
\n======================================================================
👤 j5x648@icloud.com | 订单: W1610977865 | 名字: 神原 龍一
🔄 打开订单页面...
✅ 订单页